In [ ]:
!pip install transformers pdfplumber faiss-cpu torch tk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.3 MB/s eta 0:00:00


In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 68.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
import os
import pdfplumber
import re
import json
import faiss
import numpy as np
import spacy
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

ner_pipe = pipeline("ner", model="Gherman/bert-base-NER-Russian")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [4]:


def extract_text_from_pdfs(pdf_folder, output_folder):
    """
    Извлекает текст из PDF-файлов в указанной папке и сохраняет его в текстовые файлы.

    :param pdf_folder: Путь к папке с PDF-файлами.
    :param output_folder: Путь к папке, куда будут сохранены текстовые файлы.
    """
    # Создаем папку для текстовых файлов, если она не существует
    os.makedirs(output_folder, exist_ok=True)

    # Проходим по всем PDF-файлам в папке
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            output_path = os.path.join(output_folder, pdf_file.replace(".pdf", ".txt"))

            # Извлекаем текст из PDF
            with pdfplumber.open(pdf_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

            # Сохраняем текст в файл
            with open(output_path, 'w', encoding='utf-8') as text_file:
                text_file.write(full_text)

            print(f"Текст из {pdf_file} сохранен в {output_path}")

# Пример использования
pdf_folder = "/content/pdf"  # Замените на путь к папке с PDF-файлами
output_folder = "/content/txt"  # Замените на путь к папке для текстовых файлов
extract_text_from_pdfs(pdf_folder, output_folder)


Текст из elibrary_67919195_78145120.pdf сохранен в /content/txt/elibrary_67919195_78145120.txt
Текст из elibrary_42957732_12477090.pdf сохранен в /content/txt/elibrary_42957732_12477090.txt
Текст из elibrary_67216353_50970437.pdf сохранен в /content/txt/elibrary_67216353_50970437.txt
Текст из elibrary_11606558_38965127.pdf сохранен в /content/txt/elibrary_11606558_38965127.txt
Текст из elibrary_233073_98825509.pdf сохранен в /content/txt/elibrary_233073_98825509.txt
Текст из elibrary_36936109_45147683.pdf сохранен в /content/txt/elibrary_36936109_45147683.txt
Текст из elibrary_42949376_18034799.pdf сохранен в /content/txt/elibrary_42949376_18034799.txt
Текст из elibrary_42949305_99738747.pdf сохранен в /content/txt/elibrary_42949305_99738747.txt
Текст из elibrary_233079_31627633.pdf сохранен в /content/txt/elibrary_233079_31627633.txt
Текст из elibrary_11600486_74871194.pdf сохранен в /content/txt/elibrary_11600486_74871194.txt
Текст из elibrary_42959097_39290597.pdf сохранен в /conten

In [5]:
import os
import pdfplumber
import json
import re

def is_text_quality_acceptable(text, min_length=100, max_caps_ratio=0.4, max_non_alpha_ratio=0.4):
    """
    Проверяет, удовлетворяет ли текст заданным критериям качества.

    :param text: Извлеченный текст.
    :param min_length: Минимальная длина текста для принятия.
    :param max_caps_ratio: Максимальный процент слов с заглавными буквами.
    :param max_non_alpha_ratio: Максимальный процент неалфавитных символов.
    :return: True, если текст удовлетворяет критериям, иначе False.
    """
    if len(text) < min_length:
        return False

    words = text.split()
    num_words = len(words)
    if num_words == 0:
        return False

    # Процент слов с заглавными буквами
    caps_words = [word for word in words if word.isupper()]
    caps_ratio = len(caps_words) / num_words

    # Процент неалфавитных символов
    non_alpha_chars = re.findall(r'[^а-яА-ЯёЁ\s]', text)
    non_alpha_ratio = len(non_alpha_chars) / len(text)

    # Проверка критериев
    if caps_ratio > max_caps_ratio or non_alpha_ratio > max_non_alpha_ratio:
        return False

    return True

def extract_text_from_pdfs_to_json(pdf_folder, output_json_path):
    """
    Извлекает текст из PDF-файлов в указанной папке, фильтрует его по качеству и сохраняет в JSON-файл.

    :param pdf_folder: Путь к папке с PDF-файлами.
    :param output_json_path: Путь к JSON-файлу, в который будет сохранен результат.
    """
    articles = []

    # Проходим по всем PDF-файлам в папке
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            print(f"Обработка файла: {pdf_file}")

            # Извлекаем текст из PDF
            with pdfplumber.open(pdf_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        full_text += page_text + "\n"

            # Проверка качества текста
            if is_text_quality_acceptable(full_text):
                articles.append({
                    "filename": pdf_file,
                    "text": full_text
                })
                print(f"Текст из {pdf_file} добавлен в итоговый файл.")
            else:
                print(f"Текст из {pdf_file} не прошел проверку качества и был пропущен.")

    # Сохраняем все статьи в JSON-файл
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(articles, json_file, ensure_ascii=False, indent=4)

    print(f"Все статьи сохранены в {output_json_path}")

# Пример использования
pdf_folder = "/content/pdf"  # Замените на путь к папке с PDF-файлами
output_json_path = "filtered_articles_data.json"  # Путь к выходному JSON-файлу
extract_text_from_pdfs_to_json(pdf_folder, output_json_path)


Обработка файла: elibrary_67919195_78145120.pdf
Текст из elibrary_67919195_78145120.pdf добавлен в итоговый файл.
Обработка файла: elibrary_42957732_12477090.pdf
Текст из elibrary_42957732_12477090.pdf добавлен в итоговый файл.
Обработка файла: elibrary_67216353_50970437.pdf
Текст из elibrary_67216353_50970437.pdf добавлен в итоговый файл.
Обработка файла: elibrary_11606558_38965127.pdf
Текст из elibrary_11606558_38965127.pdf добавлен в итоговый файл.
Обработка файла: elibrary_233073_98825509.pdf
Текст из elibrary_233073_98825509.pdf добавлен в итоговый файл.
Обработка файла: elibrary_36936109_45147683.pdf
Текст из elibrary_36936109_45147683.pdf добавлен в итоговый файл.
Обработка файла: elibrary_42949376_18034799.pdf
Текст из elibrary_42949376_18034799.pdf добавлен в итоговый файл.
Обработка файла: elibrary_42949305_99738747.pdf
Текст из elibrary_42949305_99738747.pdf добавлен в итоговый файл.
Обработка файла: elibrary_233079_31627633.pdf
Текст из elibrary_233079_31627633.pdf добавлен

In [7]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

def load_model():
    """
    Загружает модель и токенайзер для распознавания именованных сущностей.
    """
    model_name = "Gherman/bert-base-NER-Russian"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    return nlp

def extract_metadata(text):
    """
    Извлекает метаданные из текста с использованием регулярных выражений.

    :param text: Текст статьи.
    :return: Словарь с метаданными.
    """
    lines = text.split('\n')
    first_20_lines = " ".join(lines[:20])
    first_10_lines = " ".join(lines[:10])

    metadata = {
        "title": None,
        "authors": [],
        "year": None,
        "publisher": None
    }

    # Ищем название статьи (слова в верхнем регистре)
    potential_titles = [line.strip() for line in lines[:10] if line.isupper() and len(line.split()) > 2]
    if len(potential_titles) >= 2:
        metadata["title"] = potential_titles[0] + " " + potential_titles[1]
    elif potential_titles:
        metadata["title"] = potential_titles[0]

    # Ищем авторов (паттерн: И.И. Фамилия или Фамилия И.И.)
    author_pattern = re.compile(r"\b([А-ЯЁ][а-яё]+\s[А-ЯЁ]\.[А-ЯЁ]\.|[А-ЯЁ]\.[А-ЯЁ]\.\s[А-ЯЁ][а-яё]+)")
    authors = author_pattern.findall(first_20_lines)
    metadata["authors"] = list(set(authors))  # Убираем дубликаты, если найдены

    # Ищем год (число из 4 цифр в контексте с "г.", "год", "года")
    year_pattern = re.compile(r"\b(19|20)\d{2}\b")
    year_match = year_pattern.search(first_20_lines)
    if year_match:
        metadata["year"] = year_match.group()

    # Ищем издательство (ключевые слова и символ №)
    publisher_keywords = ["издательство", "изд-во", "публикация", "журнал", "выпуск", "№"]
    publisher_pattern = re.compile(rf"\b({'|'.join(publisher_keywords)})\b", re.IGNORECASE)
    publisher_match = publisher_pattern.search(first_20_lines)
    if publisher_match:
        metadata["publisher"] = publisher_match.group()

    return metadata

def display_metadata(json_path):
    """
    Загружает тексты статей из JSON-файла, извлекает метаданные и выводит их на экран.

    :param json_path: Путь к JSON-файлу с текстами статей.
    """
    # Загружаем статьи
    with open(json_path, 'r', encoding='utf-8') as json_file:
        articles = json.load(json_file)

    processed_articles = []

    # Обрабатываем каждую статью
    for article in articles:
        print(f"\nИзвлечение метаданных из: {article['filename']}")
        metadata = extract_metadata(article['text'])
        metadata["filename"] = article["filename"]
        processed_articles.append(metadata)
        print(f"Название: {metadata['title']}")
        print(f"Авторы: {', '.join(metadata['authors'])}")
        print(f"Год: {metadata['year']}")
        print(f"Издательство: {metadata['publisher']}")

    return processed_articles

# Пример использования
input_json_path = "/content/filtered_articles_data.json"  # Путь к JSON-файлу с текстами статей
articles_metadata = display_metadata(input_json_path)




Извлечение метаданных из: elibrary_67919195_78145120.pdf
Название: ДЕМОГРАФИЯ РЫНКА ТРУДА СТАТИСТИЧЕСКИЙ АНАЛИЗ ДИСТАНЦИОННОЙ ЗАНЯТОСТИ
Авторы: 
Год: 2024
Издательство: None

Извлечение метаданных из: elibrary_42957732_12477090.pdf
Название: SCIENCE AND TECHNOLOGY INNOVATIONS РАЗРАБОТКА ИНТЕГРИРОВАННОЙ СИСТЕМЫ УПРАВЛЕНИЯ
Авторы: 
Год: None
Издательство: None

Извлечение метаданных из: elibrary_67216353_50970437.pdf
Название: None
Авторы: 
Год: 2024
Издательство: журнал

Извлечение метаданных из: elibrary_11606558_38965127.pdf
Название: None
Авторы: Павлова С.А.
Год: 2007
Издательство: None

Извлечение метаданных из: elibrary_233073_98825509.pdf
Название: None
Авторы: 
Год: 1999
Издательство: None

Извлечение метаданных из: elibrary_36936109_45147683.pdf
Название: ИЗУЧЕНИЕ КОМПЬЮТЕРНЫХ СРЕДСТВ МАТЕМАТИЧЕСКОЙ СТАТИСТИКИ В КУРСЕ
Авторы: 
Год: 2009
Издательство: None

Извлечение метаданных из: elibrary_42949376_18034799.pdf
Название: О СРЕДСТВАХ ЗАЩИТЫ ЦИФРОВОЙ И АНАЛОГОВОЙ
Авторы: 
Год: 

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import json

def load_summarization_model():
    """
    Загружает модель ruT5 для суммаризации текста.
    """
    model_name = "IlyaGusev/rut5_base_sum_gazeta"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

def generate_annotation(text, tokenizer, model, max_length=200):
    """
    Генерирует аннотацию для статьи.

    :param text: Текст статьи.
    :param tokenizer: Токенайзер модели.
    :param model: Модель для суммаризации.
    :param max_length: Максимальная длина суммаризации.
    :return: Аннотация.
    """
    input_text = f"суммаризация: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=50, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def add_annotations_to_metadata(json_path, output_json_path, annotations_output_path):
    """
    Загружает тексты статей из JSON-файла, создает аннотации и сохраняет результат.

    :param json_path: Путь к JSON-файлу с текстами статей.
    :param output_json_path: Путь для сохранения результата с аннотациями.
    :param annotations_output_path: Путь для сохранения аннотаций в текстовом формате.
    """
    # Загружаем модель и токенайзер
    tokenizer, model = load_summarization_model()

    # Загружаем статьи
    with open(json_path, 'r', encoding='utf-8') as json_file:
        articles = json.load(json_file)

    processed_articles = []
    annotations = []  # Список для хранения аннотаций

    # Обрабатываем каждую статью
    for article in articles:
        print(f"\nСоздание аннотации для: {article['filename']}")

        # Генерируем аннотацию
        annotation = generate_annotation(article['text'], tokenizer, model)

        # Добавляем аннотацию в метаданные
        article['annotation'] = {annotation}"
        processed_articles.append(article)

        # Добавляем аннотацию в список
        annotations.append(f"{article['filename']}: {article['annotation']}")

        print(f"Аннотация: {article['annotation']}")



# Пример использования
input_json_path = "/content/filtered_articles_data.json"  # Путь к JSON-файлу с текстами статей




tokenizer_config.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]


Создание аннотации для: elibrary_67919195_78145120.pdf
Аннотация: В статье рассматривается В Российской Федерации наблюдаются значительные изменения в экономике, социальной сфере и рынке труда. Наибольшая доля работающих дистанционно на территории своего субъекта в общей численности удаленных работников характерна для субъектов Дальневосточного, Сибирского и Приволжского федеральных округов.

Создание аннотации для: elibrary_42957732_12477090.pdf
Аннотация: В статье рассматривается Интегрированная система управления объектом «Умный дом» разработана на базе парсинга Get-запросов. Система работает не только в домашней локальной сети, но и корректно функционирует при обращении к нему пользователей из глобальной сети.

Создание аннотации для: elibrary_67216353_50970437.pdf
Аннотация: В статье рассматривается Статистика и математические методы статистики способствуют развитию экономики, а также помогают экономистам и менеджерам принимать обоснованные решения и корректировать стратегии разв

IsADirectoryError: [Errno 21] Is a directory: '/content/annot'

articles = load_articles(json_path)
print(f"Загружено {len(articles)} статей.")
for article in articles[:5]:  # Выводим первые 5 статей для проверки
    print(article)


In [13]:
import json
import re
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

def load_summarization_model():
    """
    Загружает модель ruT5 для суммаризации текста.
    """
    model_name = "IlyaGusev/rut5_base_sum_gazeta"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

def generate_annotation(text, tokenizer, model, max_length=200):
    """
    Генерирует аннотацию для статьи.

    :param text: Текст статьи.
    :param tokenizer: Токенайзер модели.
    :param model: Модель для суммаризации.
    :param max_length: Максимальная длина суммаризации.
    :return: Аннотация.
    """
    input_text = f"суммаризация: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=50, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def extract_metadata(text):
    """ Извлекает метаданные: автор, название, год, издательство. """
    author = re.findall(r'([А-ЯЁ][а-яё]+(?:\s+[А-ЯЁ]\.){1,2})', text)  # Поиск авторов
    year = re.findall(r'\b(20\d{2})\b', text)  # Поиск года
    title = re.findall(r'([А-ЯЁ\s.,!?]+)', text)  # Поиск названий
    publishing = re.findall(r'издательство\s+([А-ЯЁа-яё\s.,-]+)', text)  # Поиск издательства

    return {
        "authors": author if author else ['Неизвестно'],
        "year": year[0] if year else 'Неизвестно',
        "title": title[0].strip().upper() if title else 'Неизвестно',
        "publisher": publishing[0] if publishing else 'Неизвестно'
    }

def add_annotations_to_metadata(json_path, output_json_path):
    """
    Загружает тексты статей из JSON-файла, создает аннотации и сохраняет результат.

    :param json_path: Путь к JSON-файлу с текстами статей.
    :param output_json_path: Путь для сохранения результата с аннотациями.
    """
    # Загружаем модель и токенизатор
    tokenizer, model = load_summarization_model()

    # Загружаем статьи
    with open(json_path, 'r', encoding='utf-8') as json_file:
        articles = json.load(json_file)

    processed_articles = []

    # Обрабатываем каждую статью
    for article in articles:
        print(f"\nСоздание аннотации для: {article['filename']}")

        # Извлекаем метаданные
        metadata = extract_metadata(article['text'])

        # Генерируем аннотацию
        annotation = generate_annotation(article['text'], tokenizer, model)

        # Добавляем метаданные и аннотацию к статье
        metadata["annotation"] = annotation
        metadata["filename"] = article["filename"]
        processed_articles.append(metadata)

        print(f"Название: {metadata['title']}")
        print(f"Авторы: {', '.join(metadata['authors'])}")
        print(f"Год: {metadata['year']}")
        print(f"Издательство: {metadata['publisher']}")
        print(f"Аннотация: {metadata['annotation']}")

    # Запись метаданных и аннотаций в выходной JSON-файл
    with open(output_json_path, 'w', encoding='utf-8') as output_file:
        json.dump(processed_articles, output_file, ensure_ascii=False, indent=4)

# Пример использования
input_json_path = "/content/filtered_articles_data.json"  # Путь к JSON-файлу с текстами статей
output_json_path = "articles_with_annotations.json"  # Путь для сохранения выходного файла

add_annotations_to_metadata(input_json_path, output_json_path)

print(f"Метаданные и аннотации успешно сохранены в {output_json_path}.")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Создание аннотации для: elibrary_67919195_78145120.pdf
Название: ДЕМОГРАФИЯ РЫНКА ТРУДА
Авторы: Давлетшина Л. А., Долгих Е. А.
Год: 2024
Издательство: Неизвестно
Аннотация: В Российской Федерации наблюдаются значительные изменения в экономике, социальной сфере и рынке труда. Наибольшая доля работающих дистанционно на территории своего субъекта в общей численности удаленных работников характерна для субъектов Дальневосточного, Сибирского и Приволжского федеральных округов.

Создание аннотации для: elibrary_42957732_12477090.pdf
Название: 
Авторы: Айзенберг Ю., Баев В.
Год: 2006
Издательство: Неизвестно
Аннотация: Интегрированная система управления объектом «Умный дом» разработана на базе парсинга Get-запросов. Система работает не только в домашней локальной сети, но и корректно функционирует при обращении к нему пользователей из глобальной сети.

Создание аннотации для: elibrary_67216353_50970437.pdf
Название: 
Авторы: Ходжалыева М., Нургелдиев А., Ходжалыева М., Нургелдиев А.
Год: 202

In [22]:
!pip install torch sentence-transformers


In [6]:
import gradio as gr
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка статей из JSON
def load_articles(json_path):
    """Загружает статьи из JSON-файла."""
    with open(json_path, 'r', encoding='utf-8') as file:
        articles = json.load(file)
    return articles

# Предобработка текста
def preprocess_text(text):
    """Предобработка текста: приведение к нижнему регистру."""
    return text.lower()

# Функция поиска статей
def search_articles(query, articles):
    """Ищет статьи по запросу и возвращает наиболее релевантные."""
    processed_articles = [preprocess_text(article['annotation']) for article in articles]
    processed_query = preprocess_text(query)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_articles + [processed_query])

    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    similar_indices = np.argsort(cosine_similarities)[::-1][:3]  # Топ-3 статьи

    results = []
    for idx in similar_indices:
        article = articles[idx]
        results.append({
            'title': article.get('title', 'Неизвестно'),
            'authors': article.get('authors', ['Неизвестно']),
            'year': article.get('year', 'Неизвестно'),
            'publisher': article.get('publisher', 'Неизвестно'),
            'annotation': article.get('annotation', 'Нет аннотации')
        })
    return results

# Функция для интерфейса Gradio
def gradio_search(query):
    json_path = "/content/articles_with_annotations.json"  # Путь к вашему JSON-файлу с текстами статей
    articles = load_articles(json_path)
    results = search_articles(query, articles)
    return [(result['title'], ', '.join(result['authors']), result['year'], result['publisher'], result['annotation']) for result in results]

# Создание интерфейса Gradio
iface = gr.Interface(
    fn=gradio_search,
    inputs="text",
    outputs=gr.Dataframe(headers=["Название", "Авторы", "Год", "Издательство", "Аннотация"]),
    title="Поиск статей",
    description="Введите запрос для поиска статей. Результаты поиска отображаются в таблице."
)

# Запуск интерфейса
iface.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b8f7da1a82577c5d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
!pip install gradio